In [2]:
import json
import pandas as pd

with open('../data/orgs/uk/govuk_orgs_enriched.json', 'r') as file:
    json_data = json.load(file)


# Read JSON data into a DataFrame
df = pd.read_json('../data/orgs/uk/govuk_orgs_enriched.json')

# Write DataFrame to CSV
df.to_csv('data.csv', index=False)

# break down parent-child relationships to allow graphing

In [3]:
# iot graph parent/child relationships, we want to:
# - break down the nested dict within 'parent' into 'parent_id' and 'parent_url' 
# --- actually we don't need parent_url as they should map
# - so append a new column to the df with parent_id and use that and id to generate the edges


# iterate through each row
# extract parent_id
# create a new df
# zip the two together
df['parent_id'] = df['parent_organisations'].apply(
      lambda x: x[0]['id'] if x and len(x) > 0 else None
)
df


    #print(row.parent_organisations[:].get('id'))
    # print(json_normalize(row.parent_organisations))

,id,title,format,updated_at,web_url,details,analytics_identifier,parent_organisations,child_organisations,superseded_organisations,superseding_organisations,non_govuk_domain,parent_id
0,https://www.gov.uk/api/organisations/active-tr...,Active Travel England,Executive agency,2024-10-03T13:15:29.000+01:00,https://www.gov.uk/government/organisations/ac...,"{'slug': 'active-travel-england', 'abbreviatio...",EA1350,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.activetravelengland.gov.uk,https://www.gov.uk/api/organisations/departmen...
1,https://www.gov.uk/api/organisations/advanced-...,Advanced Research and Invention Agency,Executive non-departmental public body,2022-07-19T09:24:41.000+01:00,https://www.gov.uk/government/organisations/ad...,{'slug': 'advanced-research-and-invention-agen...,PB1364,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],https://www.aria.org.uk,https://www.gov.uk/api/organisations/departmen...
2,https://www.gov.uk/api/organisations/advisory-...,Advisory Committee on Animal Feedingstuffs,Advisory non-departmental public body,2014-10-15T15:36:52.000+01:00,https://www.gov.uk/government/organisations/ad...,{'slug': 'advisory-committee-on-animal-feeding...,PB573,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.food.gov.uk/acaf,https://www.gov.uk/api/organisations/food-stan...
3,https://www.gov.uk/api/organisations/advisory-...,Advisory Committee on Novel Foods and Processes,Advisory non-departmental public body,2014-10-15T15:36:52.000+01:00,https://www.gov.uk/government/organisations/ad...,{'slug': 'advisory-committee-on-novel-foods-an...,PB574,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.food.gov.uk/acnfp,https://www.gov.uk/api/organisations/food-stan...
4,https://www.gov.uk/api/organisations/advisory-...,Advisory Committee on the Microbiological Safe...,Advisory non-departmental public body,2014-10-15T15:36:53.000+01:00,https://www.gov.uk/government/organisations/ad...,{'slug': 'advisory-committee-on-the-microbiolo...,PB575,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.food.gov.uk/acmsf,https://www.gov.uk/api/organisations/food-stan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,https://www.gov.uk/api/organisations/welsh-lan...,Welsh Language Commissioner,Other,2015-02-26T14:26:21.000+00:00,https://www.gov.uk/government/organisations/we...,"{'slug': 'welsh-language-commissioner', 'abbre...",OT1101,[],[],[],[{'id': 'https://www.gov.uk/api/organisations/...,http://www.comisiynyddygymraeg.org/,None
891,https://www.gov.uk/api/organisations/westminst...,Westminster Foundation for Democracy,Executive non-departmental public body,2020-09-02T06:02:51.000+01:00,https://www.gov.uk/government/organisations/we...,{'slug': 'westminster-foundation-for-democracy...,OT316,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.wfd.org/,https://www.gov.uk/api/organisations/foreign-c...
892,https://www.gov.uk/api/organisations/wilton-park,Wilton Park,Executive agency,2020-09-02T06:01:01.000+01:00,https://www.gov.uk/government/organisations/wi...,"{'slug': 'wilton-park', 'abbreviation': 'Wilto...",EA62,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.wiltonpark.org.uk,https://www.gov.uk/api/organisations/foreign-c...
893,https://www.gov.uk/api/organisations/yorkshire...,Yorkshire Dales National Park Authority,Other,2014-10-15T15:36:47.000+01:00,https://www.gov.uk/government/organisations/yo...,{'slug': 'yorkshire-dales-national-park-author...,OT547,[{'id': 'https://www.gov.uk/api/organisations/...,[],[],[],http://www.yorkshiredales.org.uk/,https://www.gov.uk/api/organisations/departmen...


In [4]:
# Look at all orgs with a parent
df[df['parent_id'].notna()][['parent_id','id']]

,parent_id,id
0,https://www.gov.uk/api/organisations/departmen...,https://www.gov.uk/api/organisations/active-tr...
1,https://www.gov.uk/api/organisations/departmen...,https://www.gov.uk/api/organisations/advanced-...
2,https://www.gov.uk/api/organisations/food-stan...,https://www.gov.uk/api/organisations/advisory-...
3,https://www.gov.uk/api/organisations/food-stan...,https://www.gov.uk/api/organisations/advisory-...
4,https://www.gov.uk/api/organisations/food-stan...,https://www.gov.uk/api/organisations/advisory-...
...,...,...
888,https://www.gov.uk/api/organisations/departmen...,https://www.gov.uk/api/organisations/wallace-c...
891,https://www.gov.uk/api/organisations/foreign-c...,https://www.gov.uk/api/organisations/westminst...
892,https://www.gov.uk/api/organisations/foreign-c...,https://www.gov.uk/api/organisations/wilton-park
893,https://www.gov.uk/api/organisations/departmen...,https://www.gov.uk/api/organisations/yorkshire...


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import json
import textwrap
from collections import defaultdict, deque

def parse_orgs(org_list):
    """Extract organization IDs from the organization list"""
    # Handle None/NaN
    if org_list is None:
        return []
    # If it's already a list, extract IDs
    if isinstance(org_list, list):
        if len(org_list) == 0:
            return []
        # Extract just the 'id' field from each organization dict
        return [org.get('id') for org in org_list if isinstance(org, dict) and 'id' in org]
    # Fallback for unexpected types
    return []

def wrap_text_textwrap(text, width=20):
    """Wrap text using Python's textwrap module"""
    return '<br>'.join(textwrap.wrap(text, width=width))

# Extract relationships
df['parent_list'] = df['parent_organisations'].apply(parse_orgs)
df['child_list'] = df['child_organisations'].apply(parse_orgs) 


# Create ID lookup
df['org_id'] = df['id']
id_to_title = dict(zip(df['org_id'], df['title']))
id_to_format = dict(zip(df['org_id'], df['format']))
id_to_web_url = dict(zip(df['org_id'], df['web_url']))

# Build parent-child mapping
parent_to_children = defaultdict(list)
child_to_parent = {}

for idx, row in df.iterrows():
    org_id = row['org_id']
    
    # Add children relationships - child is already an ID string, not a dict
    for child_id in row['child_list']:
        parent_to_children[org_id].append(child_id)
        child_to_parent[child_id] = org_id

# Find root nodes (organizations with no parents)
roots = [org_id for org_id in df['org_id'] if org_id not in child_to_parent]

print(f"Total organizations: {len(df)}")
print(f"Root organizations (no parents): {len(roots)}")
print(f"Organizations with children: {sum(1 for children in parent_to_children.values() if children)}")

# Print all unique formats to verify them
print(f"\nUnique organization formats:")
for fmt in sorted(df['format'].unique()):
    count = len(df[df['format'] == fmt])
    print(f"  {fmt}: {count}")

# Define format-based size weights, based on a reckoning of importance (i.e. ministerial departments are largest)
FORMAT_WEIGHTS = {
    'Ministerial department': 1000,
    'Non-ministerial department': 800,
    'Executive non-departmental public body': 600,
    'Executive agency': 600,
    'Executive office': 600,
    'Public corporation': 600,
    'Tribunal non-departmental public body': 200,
    'Independent monitoring body': 200,
    'Civil service': 200,
    'Court': 200,
    'Devolved administration': 1000,
    'Other': 200,
    'Advisory non-departmental public body': 200,
    'Sub-organisation': 200,
}

# Build tree structure for plotly treemap
# We need: labels (names), parents (parent names), and values
labels = []
parents = []
ids = []
formats = []
values = []
web_urls = []

# Add all organizations
for org_id in df['org_id']:
    # Wrap long titles for better display
    title = id_to_title.get(org_id, org_id.split('/')[-1])
    wrapped_title = wrap_text_textwrap(title, width=25)
    labels.append(wrapped_title)
    web_urls.append(id_to_web_url.get(org_id, ""))
    ids.append(org_id)
    org_format = id_to_format.get(org_id, 'Other')
    formats.append(org_format)
    
    # Set parent
    if org_id in child_to_parent:
        parent_id = child_to_parent[org_id]
        parent_title = id_to_title.get(parent_id, parent_id.split('/')[-1])
        wrapped_parent = wrap_text_textwrap(parent_title, width=25)
        parents.append(wrapped_parent)
    else:
        # Root organization - set parent to empty string
        parents.append("")
    
    # Count total descendants (BFS)
    descendants = 0
    queue = deque([org_id])
    visited = {org_id}
    while queue:
        current = queue.popleft()
        for child in parent_to_children.get(current, []):
            if child not in visited:
                visited.add(child)
                queue.append(child)
                descendants += 1
    
    # Calculate value based on format weight + bonus for number of descendants
    format_weight = FORMAT_WEIGHTS.get(org_format, 50)
    descendant_bonus = descendants * 5 
    values.append(format_weight + descendant_bonus)

# Create DataFrame for plotting
plot_df = pd.DataFrame({
    'labels': labels,
    'parents': parents,
    'ids': ids,
    'formats': formats,
    'values': values,
    'web_urls': web_urls
})

import plotly.express as px

# Professional 
plot_colors = ['#1D70B8', '#003078', '#5694CA', '#2B8CC4', '#004C8C',
              '#505a5e', '#626a6e', '#b1b4b6', '#dee0e2', '#6f777b']

# Create Treemap
fig_treemap = px.treemap(
    plot_df,
    names='labels',
    parents='parents',
    values='values',
    color='formats',
    color_discrete_sequence=plot_colors,
    custom_data=['formats','web_urls'],  
    hover_data={'formats': True, 'web_urls': True},
    title='UK Government Organizational Hierarchy',
    height=1200,
    width=1600,
)

fig_treemap.update_traces(
    textposition='top left',  # Position labels at top-left to create header effect
    textfont=dict(
        size=11,
        family='Arial, sans-serif',
        color='white'
    ),
    marker=dict(
        line=dict(color='grey', width=1),
        pad=dict(t=20, l=3, r=3, b=3)  # Add top padding for header space
    ),
    hovertemplate='<b>%{label}</b><br>Type: %{customdata[0]}<br>URL: %{customdata[1]}<extra></extra>'
)

fig_treemap.update_layout(
    font=dict(size=12, family='Arial, sans-serif'),
    margin=dict(t=100, l=10, r=10, b=10),
    paper_bgcolor='#f8f9fa',
    plot_bgcolor='#f8f9fa',
    title=dict(
        text='UK Government Organizational Hierarchy<br><sub style="font-size:13px; color:#505a5e;">Size reflects org format and descendant count</sub>',
        font=dict(size=26, color='#0b0c0c', family='Arial'),
        x=0.5,
        xanchor='center'
    ),
)

html_str = fig_treemap.to_html()


# Add JavaScript for click handling - currently doesn't work
click_script = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var plot = document.getElementsByClassName('plotly')[0];
    plot.on('plotly_click', function(data) {
        var url = data.points[0].customdata[1];
        if (url) {
            window.open(url, '_blank');
        }
    });
});
</script>
"""

html_str = html_str.replace('</body>', click_script + '</body>')
treemap_path = '../org_hierarchy_proper_treemap.html'

with open(treemap_path, 'w', encoding='utf-8') as f:
    f.write(html_str)


print(f"\nTreemap saved: {treemap_path}")


# todo - add ALBexplorer headcount numbers to scale size (GitHub)
# todo - add Institute for Government headcount stats
# todo - list of all email domains for government

Total organizations: 895
Root organizations (no parents): 794
Organizations with children: 43

Unique organization formats:
  Advisory non-departmental public body: 46
  Court: 1
  Devolved government: 3
  Executive agency: 115
  Executive non-departmental public body: 145
  Executive office: 1
  Independent monitoring body: 4
  Ministerial department: 30
  Non-ministerial department: 14
  Other: 427
  Public corporation: 22
  Special health authority: 4
  Sub organisation: 73
  Tribunal: 10

Treemap saved: ../org_hierarchy_proper_treemap.html
